In [7]:
import pandas as pd

CSV = pd.read_csv('https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r01/access/csv/ibtracs.ALL.list.v04r01.csv')
print(CSV)

C:\Users\deela\AppData\Local\Temp\ipykernel_2608\42542008.py:3: DtypeWarning: Columns (1,2,8,9,14,172,173) have mixed types. Specify dtype option on import or set low_memory=False.
  CSV = pd.read_csv('https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r01/access/csv/ibtracs.ALL.list.v04r01.csv')


                  SID SEASON NUMBER BASIN SUBBASIN     NAME  \
0                       Year                                  
1       1842298N11080   1842      1    NI       BB  UNNAMED   
2       1842298N11080   1842      1    NI       BB  UNNAMED   
3       1842298N11080   1842      1    NI       BB  UNNAMED   
4       1842298N11080   1842      1    NI       BB  UNNAMED   
...               ...    ...    ...   ...      ...      ...   
718873  2025205N13144   2025     48    WP       MM    KROSA   
718874  2025205N13144   2025     48    WP       MM    KROSA   
718875  2025205N13144   2025     48    WP       MM    KROSA   
718876  2025205N13144   2025     48    WP       MM    KROSA   
718877  2025205N13144   2025     48    WP       MM    KROSA   

                   ISO_TIME NATURE            LAT           LON  ...  \
0                                   degrees_north  degrees_east  ...   
1       1842-10-25 03:00:00     NR           10.9          80.3  ...   
2       1842-10-25 06:00:00

In [16]:
prefixes = ('TOKYO', 'TD9635', 'CMA', 'HKO', 'KMA', 'NEWDELHI', 'REUNION', 'BOM', 'NADI', 'TD9636', 'WELLINGTON', 'DS824', 'NEUMANN', 'MLC')
CSV = CSV.loc[:, ~CSV.columns.str.startswith(prefixes)]

print(CSV)

                  SID SEASON NUMBER BASIN SUBBASIN     NAME  \
0                       Year                                  
1       1842298N11080   1842      1    NI       BB  UNNAMED   
2       1842298N11080   1842      1    NI       BB  UNNAMED   
3       1842298N11080   1842      1    NI       BB  UNNAMED   
4       1842298N11080   1842      1    NI       BB  UNNAMED   
...               ...    ...    ...   ...      ...      ...   
718873  2025205N13144   2025     48    WP       MM    KROSA   
718874  2025205N13144   2025     48    WP       MM    KROSA   
718875  2025205N13144   2025     48    WP       MM    KROSA   
718876  2025205N13144   2025     48    WP       MM    KROSA   
718877  2025205N13144   2025     48    WP       MM    KROSA   

                   ISO_TIME NATURE            LAT           LON  ... USA_RMW  \
0                                   degrees_north  degrees_east  ...   nmile   
1       1842-10-25 03:00:00     NR           10.9          80.3  ...           
2  

In [6]:
# Convert each array to a Pandas Dataframe for ease of use, as well calculating additional parameters
def stormObjects(l):
    for x in range(len(l)):
        l[x] = pd.DataFrame(l[x], columns = ['ID', 'Date', 'Time', 'L', 'Status', 'Latitude', 'Longitude', 'Wind', 'MSLP', 'R34NE', 'R34SE', 'R34NW', 'R34SW'])        
        l[x] = l[x].drop(l[x][l[x].Time.astype(int) % 600 != 0].index)
        l[x] = l[x].drop('L', axis = 1)
        l[x].index = range(len(l[x]))
        l[x].Latitude = ((l[x].Latitude).str[:-1]).astype(float)
        l[x].Longitude = ((l[x].Longitude).str[:-1]).astype(float) * -1
        l[x].Wind = (l[x].Wind).astype(int)
        l[x].MSLP = (l[x].MSLP).astype(int)
        l[x].Status = (l[x].Status).str.strip()
        l[x].R34NE = (l[x].R34NE).astype(int)
        l[x].R34SE = (l[x].R34SE).astype(int)
        l[x].R34NW = (l[x].R34NW).astype(int)
        l[x].R34SW = (l[x].R34SW).astype(int)

        change = []
        ri = []
        r34 = []
        speed = []
        for y in range(len(l[x]['Wind'])):
            l[x]['ID'][y] = f'{l[x]["ID"][y][0:4]}{l[x]["Date"][y][0:4]}'
            l[x]['Time'][y] = np.datetime64(f'{l[x]["Date"][y][0:4]}-{l[x]["Date"][y][4:6]}-{l[x]["Date"][y][6:8]}T{l[x]["Time"][y][1:3]}')
            try:
                if l[x]['Wind'][y] > 34:
                    r34.append((l[x]['R34NE'][y] + l[x]['R34SE'][y] + l[x]['R34NW'][y] + l[x]['R34SW'][y]) / 4)
                else:
                    r34.append(0)
                
                if (y - 4 >= 0) and ((l[x]['Wind'][y] - l[x]['Wind'][y - 4]) >= 30):
                    ri.append(True)
                else:
                    ri.append(False)
                
                if (y - 1) >= 0:
                    d = helper.distance([l[x]['Latitude'][y], l[x]['Longitude'][y]], [l[x]['Latitude'][y - 1], l[x]['Longitude'][y - 1]])
                    speed.append(round(d / 6, 1))
                else:
                    speed.append(np.nan)

                if (y - 4) >= 0:
                    change.append(l[x]['Wind'][y] - l[x]['Wind'][y - 4])
                else:
                    change.append(np.nan)
            except:
                speed.append(np.nan)
                change.append(0)
                ri.append(False)

        l[x] = l[x].drop('Date', axis = 1)
        l[x]['Speed'] = speed
        l[x]['24hrChange'] = change
        l[x]['RI'] = ri
        l[x]['R34'] = r34
        l[x]['VR'] = l[x]['Wind'] / l[x]['R34']
        l[x]['ACE'] = helper.ACE(l[x]['Wind'], l[x]['Status'])
    return l

def database():
    storms = getData(np.arange(yearsUsed[0], yearsUsed[1]))
    names, ID, years = getStormInfo(np.arange(yearsUsed[0], yearsUsed[1]))
    storms = stormObjects(storms)

    data = []
    for x in range(len(storms)):
        tempDict = {'Name' : names[x],
                    'Year' : years[x],
                    'ID' : ID[x],
                    'VMax' : storms[x]['Wind'].max(),
                    'MSLP' : storms[x]['MSLP'].min(),
                    'ACE' : round(storms[x]['ACE'].iloc[-1], 2),
                    'RI' : storms[x]['RI'].max(),
                    'Max24hr' : storms[x]['24hrChange'].max(),
                    'aceChart' : charts.acecum,
                    'track' : charts.track,
                    'ratio' : charts.VRmslp,
                    'Storm Data' : storms[x]}

        data.append(tempDict)
    return data

def retrieveStorm(CSV, storm = None, ID = None):
    data = CSV[((CSV.NAME == storm[0]) & (CSV.SEASON == storm[1])) | (CSV.USA_ATCF_ID == ID)]

    stormData = 

    tempDict = {'Storm Data' : stormData}

    for x in range(len(dataset)):
        if (dataset[x]['Name'] == storm[0].upper()) and (dataset[x]['Year'] == storm[1]):
            data = dataset[x]
            break 
        elif (dataset[x]['ID'].lower() == ID) or (dataset[x]['ID'].upper() == ID):
            data = dataset[x]
            break 
        else:
            data = 'Storm not found!'
    return data


0                 
1                 
2                 
3                 
4                 
            ...   
718873    WP122025
718874    WP122025
718875    WP122025
718876    WP122025
718877    WP122025
Name: USA_ATCF_ID, Length: 718878, dtype: object


In [14]:
def retrieveStorm(CSV, storm = None, ID = None):
    data = CSV[((CSV.NAME.str.lower() == storm[0].lower()) & (CSV.SEASON.astype(str) == str(storm[1]))) | (CSV.USA_ATCF_ID.astype(str) == str(ID))]
    return data 

In [21]:
data = retrieveStorm(CSV, ['Irma', 2017])
data.ISO_TIME = pd.to_datetime(data.ISO_TIME)
print(data.ISO_TIME)

669178   2017-08-30 00:00:00
669179   2017-08-30 03:00:00
669180   2017-08-30 06:00:00
669181   2017-08-30 09:00:00
669182   2017-08-30 12:00:00
                 ...        
669296   2017-09-13 00:00:00
669297   2017-09-13 03:00:00
669298   2017-09-13 06:00:00
669299   2017-09-13 09:00:00
669300   2017-09-13 12:00:00
Name: ISO_TIME, Length: 123, dtype: datetime64[ns]


C:\Users\deela\AppData\Local\Temp\ipykernel_2608\1092313710.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.ISO_TIME = pd.to_datetime(data.ISO_TIME)
